In [1]:
using Printf
using Random
using LinearAlgebra
using PlotlyJS
rng = MersenneTwister(18);

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/f562d6507acc94a42cdf3b074249b311a179837e-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/f562d6507acc94a42cdf3b074249b311a179837e-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
n = 3 # In 3D

Q = rand(n,n)
Q,R = qr(Q)
Λ = diagm(0 => Float32[2.0^(-i) for i=0:n-1])
A = Q * Λ * Q'
# The e-vectors are the columns of Q.

Diagonal(Λ)

3×3 Diagonal{Float32,Array{Float32,1}}:
 1.0   ⋅    ⋅  
  ⋅   0.5   ⋅  
  ⋅    ⋅   0.25

In [3]:
μ0 = Λ[1,1]
μ = μ0 * 0.8
# Our initial approximation of an eigenvalue of interest

println("Exact answer: ", μ0)
println("Initial guess: ", μ)

qk = rand(rng, n)
qk /= norm(qk)

c_fit = 5
error_old = -1

for k=1:4
    zk = (A - μ * I) \ qk
    qk = zk/norm(zk)      # Normalize e-vector
    μ = dot(qk, A * qk)   # E-value approximation
    @printf "Iteration: %1d | e-value: %10.8f | error: %7.1e " k μ abs(μ-μ0)
    if k>1
        @printf " (estim. %7.1e)\n" c_fit*error_old^3
    else
        @printf "\n"
    end
    error_old = abs(μ-μ0)
end

Exact answer: 1.0
Initial guess: 0.8
Iteration: 1 | e-value: 0.98858940 | error: 1.1e-02 
Iteration: 2 | e-value: 0.99999553 | error: 4.5e-06  (estim. 7.4e-06)
Iteration: 3 | e-value: 1.00000000 | error: 8.9e-16  (estim. 4.5e-16)
Iteration: 4 | e-value: 1.00000000 | error: 3.3e-16  (estim. 3.5e-45)


In [4]:
# Let's plot the Rayleigh quotient to observe that grad r = 0
# at all e-values.

N = 32
u = LinRange(0, 2π, N)
v = LinRange(0, π, N)

x = cos.(u) * sin.(v)'
y = sin.(u) * sin.(v)'
z = repeat(cos.(v)',outer=[N, 1])

r = zeros(N,N)
for i=1:N, j=1:N
    # The point q is on the surface of the unit sphere
    q = [x[i,j] y[i,j] z[i,j]]
    # Rayleigh-quotient at this point
    r[i,j] = dot(q, A * transpose(q))
end

P = 1.1*vcat(Q',-Q') # Increase the radius a bit

6×3 Array{Float64,2}:
 -0.806872  -0.721094  -0.197435
  0.735603  -0.713801  -0.399219
  0.133586  -0.424866   1.0058  
  0.806872   0.721094   0.197435
 -0.735603   0.713801   0.399219
 -0.133586   0.424866  -1.0058  

In [5]:
t = scatter3d(x=P[:,1], y=P[:,2], z=P[:,3],
    mode="markers", marker_size=10)

s = surface(x=x, y=y, z=z, surfacecolor=r)

l = Layout(title="Rayleigh quotient", 
    aspectratio=attr(x=1, y=1, z=1),
    scene_dragmode="orbit")

plot([s, t], l)

data: [
  "surface with fields surfacecolor, type, x, y, and z",
  "scatter3d with fields marker, mode, type, x, y, and z"
]

layout: "layout with fields aspectratio, margin, scene, and title"